# Lab 11.2: Adaptive Radiation Simulator
## Chapter 11: The Innovation Engine

### 🎯 Learning Objectives
- Model explosive diversification from single ancestor
- Calculate niche overlap and competition
- Analyze beak evolution in Darwin's finches
- Simulate rapid speciation events
- Predict radiation outcomes for new environments

### 📖 Connection to Chapter 11
This lab integrates **Section 11.3: Adaptive Radiations**:
- One ancestor, many descendants
- Darwin's finches (Galápagos)
- 15-fold beak size variation
- Ecological speciation
- Medium ground finch evolution
- Cichlid fish rapid radiation
- Conditions promoting radiation

### 🐦 The Question
**How did one finch species become 14 different species in <1 million years?**  
And can we predict when adaptive radiation will occur? Let's simulate it!

In [ ]:
# === GOOGLE COLAB SETUP ===
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print("✓ Widgets enabled")
except:
    print("✓ Running outside Colab")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import *
from IPython.display import display, clear_output
from datetime import datetime

print("✓ Ready!")

## Part 1: Adaptive Radiation Theory

### What Is Adaptive Radiation?

**Definition**: Single ancestral species → Multiple descendant species, each adapted to different ecological niche

**Classic example**: Darwin's finches
- 1 ancestral finch arrives on Galápagos
- Evolves into 14 species
- Time: <1 million years (FAST!)

### Conditions for Radiation

**1. Ecological Opportunity**
- Empty niches available
- Low competition
- Novel environment
- Key innovation unlocks resources

**2. Genetic/Developmental Factors**
- Phenotypic plasticity
- Modular development
- Few genes with large effects

### The Radiation Process

**Step 1: Colonization**
- Ancestral species arrives
- Finds empty niches

**Step 2: Diversification**
- Different populations exploit different resources
- Morphology diverges (beaks, body size)
- Competition drives separation

**Step 3: Speciation**
- Ecological differences → reproductive isolation
- Song differences, mating preferences
- New species formed

### Niche Overlap Formula

**Competition strength**:

$$C = e^{-\alpha (d_{beak})^2}$$

Where:
- C = competition coefficient (0-1)
- α = niche width parameter
- d_beak = difference in beak size

**High overlap (similar beaks) → strong competition → selection for divergence**

### Fitness in Radiation

$$F = \frac{r_{max}}{1 + \sum C_i N_i}$$

Where:
- r_max = maximum growth rate
- C_i = competition with species i
- N_i = abundance of competitor i

**Fitness decreases with competition → drives niche separation**

### From Chapter 11

**Darwin's finches**:
- 15-fold beak size variation
- Seed specialists, insect specialists, nectar feeders, tool users
- Medium ground finch: Beak evolved 5% in ONE GENERATION (1977 drought)

**Cichlid fish**:
- 700+ species in African lakes
- <1 million years
- Even faster radiation!

In [ ]:
# Functions
def calc_competition(beak_diff, alpha=0.5):
    """Calculate competition between species based on beak difference"""
    return np.exp(-alpha * beak_diff**2)

def calc_fitness(beak_size, seed_sizes, competitors, r_max=1.0):
    """
    Calculate fitness based on resource match and competition
    
    beak_size: focal species beak
    seed_sizes: available seed types
    competitors: list of (competitor_beak, abundance) tuples
    """
    # Resource match (best at seeds close to beak size)
    resource = np.sum([np.exp(-0.5*(beak_size - seed)**2) for seed in seed_sizes])
    
    # Competition penalty
    competition = sum([calc_competition(abs(beak_size - comp_beak)) * abund 
                      for comp_beak, abund in competitors])
    
    # Fitness
    fitness = r_max * resource / (1 + competition)
    return fitness

def optimal_beak_for_seeds(seed_sizes):
    """Find optimal beak size for given seed distribution"""
    beaks = np.linspace(0.5, 3.0, 100)
    fitnesses = [calc_fitness(b, seed_sizes, []) for b in beaks]
    optimal = beaks[np.argmax(fitnesses)]
    return optimal

# Test
print("EXAMPLES:")
print("="*60)

# Similar beaks → high competition
diff = 0.2
comp = calc_competition(diff)
print(f"Similar beaks (diff={diff}):")
print(f"  Competition: {comp:.2f} (HIGH!)\n")

# Different beaks → low competition
diff = 2.0
comp = calc_competition(diff)
print(f"Different beaks (diff={diff}):")
print(f"  Competition: {comp:.2f} (LOW)\n")

# Fitness with competitors
beak = 1.5
seeds = [1.0, 1.5, 2.0]
competitors = [(1.4, 0.5), (2.0, 0.3)]  # (beak, abundance)
fit = calc_fitness(beak, seeds, competitors)
print(f"Fitness calculation:")
print(f"  Beak: {beak}")
print(f"  Seeds: {seeds}")
print(f"  Competitors: {len(competitors)}")
print(f"  Fitness: {fit:.2f}")

print("\n✓ Ready!")

## Part 2: Darwin's Finches Database

In [ ]:
# 14 Darwin's finch species
darwins_finches = {
    'Large Ground Finch': {
        'beak_depth_mm': 15.0, 'body_mass_g': 36, 'diet': 'Large/hard seeds',
        'island': 'Multiple', 'niche': 'Large seed specialist'
    },
    'Medium Ground Finch': {
        'beak_depth_mm': 10.0, 'body_mass_g': 21, 'diet': 'Medium seeds',
        'island': 'Multiple', 'niche': 'Generalist'
    },
    'Small Ground Finch': {
        'beak_depth_mm': 7.0, 'body_mass_g': 14, 'diet': 'Small seeds',
        'island': 'Multiple', 'niche': 'Small seed specialist'
    },
    'Sharp-beaked Ground Finch': {
        'beak_depth_mm': 8.0, 'body_mass_g': 20, 'diet': 'Seeds + blood',
        'island': 'Wolf', 'niche': 'Opportunistic (vampire!)'
    },
    'Cactus Finch': {
        'beak_depth_mm': 11.0, 'body_mass_g': 24, 'diet': 'Cactus + seeds',
        'island': 'Multiple', 'niche': 'Cactus specialist'
    },
    'Large Cactus Finch': {
        'beak_depth_mm': 13.0, 'body_mass_g': 30, 'diet': 'Large cactus',
        'island': 'Española', 'niche': 'Large cactus specialist'
    },
    'Vegetarian Finch': {
        'beak_depth_mm': 12.0, 'body_mass_g': 35, 'diet': 'Leaves/buds',
        'island': 'Multiple', 'niche': 'Folivore (unique!)'
    },
    'Woodpecker Finch': {
        'beak_depth_mm': 9.0, 'body_mass_g': 20, 'diet': 'Insects from bark',
        'island': 'Multiple', 'niche': 'Tool user!'
    },
    'Mangrove Finch': {
        'beak_depth_mm': 8.5, 'body_mass_g': 19, 'diet': 'Mangrove insects',
        'island': 'Isabela', 'niche': 'Mangrove specialist'
    },
    'Small Tree Finch': {
        'beak_depth_mm': 8.0, 'body_mass_g': 15, 'diet': 'Small insects',
        'island': 'Multiple', 'niche': 'Insectivore'
    },
    'Medium Tree Finch': {
        'beak_depth_mm': 9.5, 'body_mass_g': 18, 'diet': 'Medium insects',
        'island': 'Floreana', 'niche': 'Insectivore'
    },
    'Large Tree Finch': {
        'beak_depth_mm': 12.0, 'body_mass_g': 28, 'diet': 'Large insects',
        'island': 'Multiple', 'niche': 'Large insectivore'
    },
    'Warbler Finch': {
        'beak_depth_mm': 5.0, 'body_mass_g': 8, 'diet': 'Tiny insects',
        'island': 'Multiple', 'niche': 'Warbler-like (smallest!)'
    },
    'Cocos Finch': {
        'beak_depth_mm': 9.0, 'body_mass_g': 19, 'diet': 'Seeds + insects',
        'island': 'Cocos', 'niche': 'Isolated island generalist'
    }
}

# Calculate ranges
beaks = [f['beak_depth_mm'] for f in darwins_finches.values()]
masses = [f['body_mass_g'] for f in darwins_finches.values()]

beak_range = max(beaks) / min(beaks)
mass_range = max(masses) / min(masses)

print("DARWIN'S FINCHES DATABASE")
print("="*80)
print(f"Total species: {len(darwins_finches)}")
print(f"Beak depth range: {min(beaks):.1f} - {max(beaks):.1f} mm ({beak_range:.1f}× variation!)")
print(f"Body mass range: {min(masses)} - {max(masses)} g ({mass_range:.1f}× variation)")
print("\nSPECIES LIST:")
print("="*80)
print(f"{'Species':<30}{'Beak (mm)':<12}{'Mass (g)':<12}{'Niche'}")
print("="*80)

for name in sorted(darwins_finches.keys(), 
                  key=lambda x: darwins_finches[x]['beak_depth_mm']):
    f = darwins_finches[name]
    print(f"{name:<30}{f['beak_depth_mm']:<12}{f['body_mass_g']:<12}{f['niche']}")

print("="*80)
print("✓ Database ready!")

## Part 3: Radiation Simulator

In [ ]:
def simulate_radiation(num_niches, competition_strength):
    """
    Simulate adaptive radiation
    
    num_niches: number of available ecological niches
    competition_strength: how strongly species compete
    """
    # Start with ancestral species
    ancestral_beak = 1.0
    
    # Create seed distribution (niches)
    seed_sizes = np.linspace(0.5, 2.5, num_niches)
    
    # Evolve species to fill niches
    evolved_beaks = []
    for seed in seed_sizes:
        # Each niche selects for matching beak
        evolved_beaks.append(seed * 0.9)  # Slight offset
    
    # Calculate competition matrix
    n_species = len(evolved_beaks)
    comp_matrix = np.zeros((n_species, n_species))
    for i in range(n_species):
        for j in range(n_species):
            if i != j:
                diff = abs(evolved_beaks[i] - evolved_beaks[j])
                comp_matrix[i,j] = calc_competition(diff, alpha=competition_strength)
    
    # Plot
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Radiation Timeline', 'Beak Size Distribution',
                       'Niche Overlap', 'Competition Matrix'),
        specs=[[{'type': 'scatter'}, {'type': 'bar'}],
               [{'type': 'scatter'}, {'type': 'heatmap'}]]
    )
    
    # 1. Timeline - show radiation process
    time_points = np.linspace(0, 1, n_species)
    fig.add_trace(go.Scatter(
        x=time_points, y=evolved_beaks,
        mode='markers+lines',
        marker=dict(size=10, color='#3498DB'),
        showlegend=False
    ), row=1, col=1)
    fig.add_hline(y=ancestral_beak, line_dash='dash', 
                 annotation_text='Ancestor', row=1, col=1)
    
    # 2. Distribution
    species_names = [f"Sp {i+1}" for i in range(n_species)]
    fig.add_trace(go.Bar(
        x=species_names, y=evolved_beaks,
        marker_color='#2ECC71',
        showlegend=False
    ), row=1, col=2)
    
    # 3. Niche space
    fig.add_trace(go.Scatter(
        x=seed_sizes, y=[1]*len(seed_sizes),
        mode='markers', name='Seeds',
        marker=dict(size=15, color='#F39C12', symbol='diamond')
    ), row=2, col=1)
    fig.add_trace(go.Scatter(
        x=evolved_beaks, y=[1.5]*len(evolved_beaks),
        mode='markers', name='Beaks',
        marker=dict(size=15, color='#3498DB', symbol='circle')
    ), row=2, col=1)
    
    # 4. Competition heatmap
    fig.add_trace(go.Heatmap(
        z=comp_matrix, x=species_names, y=species_names,
        colorscale='Reds', showscale=False
    ), row=2, col=2)
    
    fig.update_xaxes(title_text="Time", row=1, col=1)
    fig.update_yaxes(title_text="Beak Size", row=1, col=1)
    fig.update_yaxes(title_text="Beak Size", row=1, col=2)
    fig.update_xaxes(title_text="Resource Size", row=2, col=1)
    fig.update_yaxes(visible=False, row=2, col=1)
    
    fig.update_layout(height=700, title_text='<b>Adaptive Radiation Simulation</b>')
    
    # Summary
    print("\nRADIATION SUMMARY")
    print("="*70)
    print(f"Ancestral beak: {ancestral_beak:.2f} mm")
    print(f"Number of niches: {num_niches}")
    print(f"Species evolved: {n_species}")
    print(f"Beak range: {min(evolved_beaks):.2f} - {max(evolved_beaks):.2f} mm")
    print(f"Fold variation: {max(evolved_beaks)/min(evolved_beaks):.1f}×")
    avg_comp = np.mean(comp_matrix[comp_matrix > 0])
    print(f"Average competition: {avg_comp:.2f}")
    print("\n→ More niches = more species = greater diversity!")
    print("="*70)
    
    fig.show()

# Interactive controls
niche_slider = IntSlider(value=7, min=3, max=15, description='Niches:')
comp_slider = FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, 
                         description='Competition:')

display(HTML("<h3>🐦 Adaptive Radiation Simulator</h3>"))
display(HTML("<p>Adjust niches and competition to see radiation patterns:</p>"))
interact(simulate_radiation, num_niches=niche_slider, 
        competition_strength=comp_slider);

## Part 4: Challenges

### Challenge 1: The 1977 Drought 🌵

**Question**: How much did medium ground finch beaks evolve in ONE generation?

**1976 Galápagos Drought**:
- Severe drought kills 85% of population
- Small seeds disappear
- Only large, hard seeds remain
- Birds with small beaks can't crack them

**Data**:
- Before drought: Average beak depth = 9.5 mm
- Survivors: Average beak depth = 10.0 mm
- Next generation: Average beak depth = 9.8 mm

**Calculate**: 
1. Selection differential (survivors - population)
2. Response to selection (offspring - parents)
3. Heritability estimate

<details>
<summary>Solution</summary>

**Selection differential (S)**:
- S = survivors - before = 10.0 - 9.5 = 0.5 mm
- This is the immediate effect of selection

**Response to selection (R)**:
- R = offspring - parents = 9.8 - 9.5 = 0.3 mm
- This is the evolutionary response

**Heritability (h²)**:
- h² = R / S = 0.3 / 0.5 = 0.6
- 60% of variation is heritable!

**What this means**:
- 5% increase in ONE generation! (0.5/9.5)
- High heritability = rapid evolution possible
- Evolution observable in real-time

**Adaptive radiation requires**:
- Heritable variation (✓)
- Selection pressure (✓)
- Empty niches (✓)
- Time (millions of years)

**Lesson**: Evolution is FAST when conditions are right!
</details>

### Challenge 2: Cichlid Speed Record 🐟

**Question**: How can 700+ species evolve in <1 million years?

**African Great Lakes**:
- Lake Victoria: 500+ endemic cichlid species
- Lake formed: ~1 million years ago
- Time per species: 1,000,000 / 500 = 2,000 years!

**Compare to mammals**:
- Typical mammal speciation: 1-10 million years
- Cichlids: 500× FASTER!

**Why so fast?**

<details>
<summary>Solution</summary>

**Factors enabling rapid radiation**:

**1. Ecological opportunity (HUGE)**
- New lake = empty niches
- No competitors
- Diverse habitats (rocky, sandy, open water)

**2. Modular development**
- Jaw bones develop independently
- Can evolve feeding without affecting body
- Pharyngeal jaws = innovation!

**3. Sexual selection**
- Female color preferences
- Rapid speciation through mate choice
- No ecological difference needed

**4. Short generation time**
- Breed at 6-12 months
- vs mammals at 2-5 years
- More generations = faster evolution

**Math**:
- 2,000 years / 1 year per generation = 2,000 generations
- With h²=0.5, traits can change substantially
- Response per generation: R = h² × S

**Key insight**: Adaptive radiation SPEED depends on:
- Ecological opportunity (more niches = faster)
- Generation time (shorter = faster)
- Developmental modularity (independent traits = faster)
- Sexual selection (adds extra speciation axis)

**Cichlids have ALL of these!**
</details>

### Challenge 3: Design Your Island 🏝️

**Scenario**: New volcanic island forms
- Size: 100 km²
- Habitats: Beach, forest, mountain (3 zones)
- One finch species arrives

**Design the radiation**:
1. How many species will evolve?
2. What beak types?
3. How long will it take?

<details>
<summary>Solution</summary>

**Predict species diversity**:

**Available niches**:
- Beach: Soft seeds (1 niche)
- Forest: Multiple seed sizes (3 niches)
- Forest: Insects (2 niches)
- Mountain: Nectar (1 niche)
- **Total: ~7 niches**

**Expected species**: 5-8 species
- Not all niches may be filled
- Some competition overlap

**Beak types**:
- **Small ground finch** (beach, small seeds): 7mm
- **Medium ground finch** (forest, medium seeds): 10mm
- **Large ground finch** (forest, large seeds): 14mm
- **Insectivore** (forest, insects): 8mm, thin
- **Nectar feeder** (mountain, flowers): 9mm, curved

**Timeline**:
- Initial diversification: 10,000-50,000 years
- Depends on: mutation rate, selection strength, migration
- Faster if: Strong selection, high heritability, no gene flow

**Limiting factors**:
- Island size (100 km² = small, limits diversity)
- Only 3 habitat types (limits niches)
- One colonization event (limits variation)

**Prediction confidence**:
- High: 5-8 species WILL evolve
- Medium: Exact beak sizes (depends on seeds)
- Low: Exact timing (many variables)

**Lesson**: Adaptive radiation is PREDICTABLE!
- Empty niches → will be filled
- Trait diversity → matches resource diversity
- Time scale → tens of thousands of years
</details>

## Part 5: Export

In [ ]:
def export_results():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = "/content"
    
    data = []
    for name, f in darwins_finches.items():
        data.append({
            'Species': name,
            'Beak_Depth_mm': f['beak_depth_mm'],
            'Body_Mass_g': f['body_mass_g'],
            'Diet': f['diet'],
            'Niche': f['niche'],
            'Island': f['island']
        })
    
    df = pd.DataFrame(data).sort_values('Beak_Depth_mm')
    csv_file = f"{output_dir}/lab_11_2_finches_{timestamp}.csv"
    df.to_csv(csv_file, index=False)
    print(f"✓ Saved: {csv_file}")
    print("\nDownload: 📁 → /content → right-click")

btn = Button(description='📥 Export', button_style='success', icon='download')
btn.on_click(lambda b: export_results())
display(HTML("<h3>📤 Export</h3>"))
display(btn)

## Summary

### Key Insights

✅ **Adaptive radiation** - One ancestor → many descendants  
✅ **Ecological opportunity** - Empty niches drive diversification  
✅ **Rapid evolution** - Observable in real-time (1977 drought)  
✅ **Predictable patterns** - Resources shape trait diversity  
✅ **Competition drives divergence** - Similar species separate  

### Darwin's Finches

**14 species from 1 ancestor!**
- **15-fold beak variation** (5mm warbler → 15mm ground finch)
- **4.5-fold mass variation** (8g → 36g)
- **Time**: <1 million years
- **Mechanism**: Ecological speciation

**Niches filled**:
- Large seed specialists
- Small seed specialists  
- Insect specialists
- Nectar feeders
- Tool users!
- Vampire (blood drinker)!
- Vegetarian (leaves)

### Speed Records

**Cichlid fish**: 700+ species in <1 million years
- Fastest known vertebrate radiation
- One new species every ~2,000 years!

**Medium ground finch**: 5% beak change in ONE generation
- 1977 drought selection
- Evolution in real-time
- High heritability (60%)

### Conditions for Radiation

**Required**:
1. Ecological opportunity (empty niches)
2. Heritable variation
3. Selection pressure
4. Time (but not much!)

**Accelerators**:
- Short generation time
- Modular development
- Sexual selection
- Key innovations

### The Big Lesson

**Evolution is both creative AND predictable!**
- Empty niches WILL be filled
- Trait diversity matches resource diversity
- Same conditions → same outcomes

**Adaptive radiation = Evolution's creativity engine!**

**Congratulations!** 🎉